In [ ]:
!pip install tensorflow

In [ ]:
!rm -rf /content/id_docs_resized
!rm -rf /content/my_dataset/

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
import os
import shutil

In [ ]:
!apt-get install -y unrar
!unrar x /content/id_docs_resized_224_nic.rar /content/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/id_docs_resized_224_nic.rar

Creating    /content/id_docs_resized                                  OK
Creating    /content/id_docs_resized/train                            OK
Creating    /content/id_docs_resized/train/old_nic                    OK
Extracting  /content/id_docs_resized/train/old_nic/image_1.jpg             0%  OK 
Extracting  /content/id_docs_resized/train/old_nic/image_10.jpeg           0%  OK 
Extracting  /content/id_docs_resized/train/old_nic/image_100.jpg           1%  OK 
Extracting  /content/id_docs_resized/train/old_nic/image_101.jpeg          2%  OK 
Extracting  /content/id_docs_resized/train/old_nic/image_102.jpe

In [ ]:
DATASET_PATH = 'id_docs_resized'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 0.0001
VALIDATION_SPLIT = 0.2

In [ ]:
# Check if the dataset path exists
if not os.path.exists(DATASET_PATH):
    print(f"Error: Dataset directory '{DATASET_PATH}' not found.")
    print("Please create the directory with a 'train' and 'validation' subfolder,")
    print("and place your images in category-specific subfolders.")
    exit()

In [ ]:
# --- Data Augmentation and Loading ---
# We use data augmentation to create more training examples from a small dataset.
# This helps the model generalize better and reduces overfitting.
train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

In [ ]:
validation_datagen = image.ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the training and validation data from the directory
print("Loading training data...")
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Loading training data...
Found 138 images belonging to 1 classes.


In [ ]:
print("\nLoading validation data...")
validation_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Loading validation data...
Found 34 images belonging to 1 classes.


In [ ]:
# Get the number of classes from the generator
num_classes = train_generator.num_classes
class_names = list(train_generator.class_indices.keys())
print(f"\nFound {num_classes} classes: {class_names}")



Found 1 classes: ['old_nic']


In [ ]:
# --- Build the Model with Transfer Learning ---
# Load the pre-trained MobileNetV2 model without the top classification layers.
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze the convolutional base
base_model.trainable = False
print("\nBase MobileNetV2 model loaded and frozen.")


Base MobileNetV2 model loaded and frozen.


In [ ]:
# Create the new model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduces the spatial dimensions of the feature maps
    Dense(512, activation='relu'),
    Dropout(0.5), # Add dropout to prevent overfitting
    Dense(num_classes, activation='softmax') # The final classification layer
])

In [ ]:
# --- Compile and Train the Model ---
print("\nCompiling model...")
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Compiling model...


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,914,369 (11.12 MB)

 Trainable params: 656,385 (2.50 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
print("\nTraining model...")
history = model.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


Training model...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 35s 6s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/20
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 1.0000 - loss: 0.0000e+00

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 785ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 644ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 556ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 724ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 811ms/step - accuracy: 1.0000 - lo

In [ ]:
# --- Save the Model ---
model.save('image_classifier_transfer_learning.h5')
print("\nModel saved to 'image_classifier_transfer_learning.h5'")


Model saved to 'image_classifier_transfer_learning.h5'


In [ ]:
# --- Predict a new image (Example) ---
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    img_array = img_array / 255.0 # Rescale the pixel values

    predictions = model.predict(img_array)
    predicted_class_index = tf.argmax(predictions[0])
    predicted_class = class_names[predicted_class_index]
    confidence = tf.reduce_max(predictions[0]) * 100

    print(f"\nPrediction for {img_path}:")
    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence.numpy():.2f}%")

In [ ]:
predict_image(model, 'cake.jpg')

/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step

Prediction for cake.jpg:
Predicted class: old_nic
Confidence: 100.00%


In [ ]:
predict_image(model, 'id_docs_resized/validation/new_nic/image_11.jpg')
predict_image(model, 'id_docs_resized/validation/new_nic/image_12.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Prediction for id_docs_resized/validation/new_nic/image_11.jpg:
Predicted class: new_nic
Confidence: 55.08%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Prediction for id_docs_resized/validation/new_nic/image_12.jpg:
Predicted class: new_nic
Confidence: 42.98%


In [ ]:
predict_image(model, 'id_docs_resized/validation/old_nic/image_41.jpg')
predict_image(model, 'id_docs_resized/validation/old_nic/image_42.jpeg')
predict_image(model, 'id_docs_resized/validation/old_nic/image_43.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Prediction for id_docs_resized/validation/old_nic/image_41.jpg:
Predicted class: old_nic
Confidence: 97.25%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Prediction for id_docs_resized/validation/old_nic/image_42.jpeg:
Predicted class: old_nic
Confidence: 97.06%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

Prediction for id_docs_resized/validation/old_nic/image_43.jpeg:
Predicted class: old_nic
Confidence: 98.34%


In [ ]:
predict_image(model, 'id_docs_resized/validation/passport/image_10.jpg')
predict_image(model, 'id_docs_resized/validation/passport/image_9.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Prediction for id_docs_resized/validation/passport/image_10.jpg:
Predicted class: passport
Confidence: 82.59%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

Prediction for id_docs_resized/validation/passport/image_9.jpg:
Predicted class: passport
Confidence: 63.50%


In [1]:
#random image
predict_image(model, 'cake.jpg')

NameError: name 'predict_image' is not defined